In [37]:
import meshio
import os
import numpy as np
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.base.base import Struct

dim = 3
float_precision = 'float32'
PATH = "/home/aben-ham/Desktop/work/manapy0.4/tests/domain/primary/3D/"
PATH = os.path.join(PATH, 'mesh', 'tetrahedron.msh')
#PATH = os.path.join(PATH, 'mesh', 'cuboid.msh')

running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
MeshPartition(PATH, dim=dim, conf=running_conf, periodic=[0, 0, 0])
domain = Domain(dim=dim, conf=running_conf)
m_mesh = meshio.read(PATH)

(int32[:,:], int32[:,:], int32[:,:])
Reading gmsh file ...
Saving partition files ...
Number of Cells: 6000
Number of Vertices: 1331
Local domain contruction ...
(int32[:,:], float64[:,:], int32, float64[:,:], float64[:])
(int32[:,:], float64[:,:], int32, int32)
(int32[:,:], int32, int32[:,:], int32[:,:])
(int32, int64[:], int32[:,:], int32[:,:], int32)
(int32[:,:], int32, int32, int32[:,:], int32)
(int32[:,:], int32[:,:], int32, int32)
(int32[:,:], int32[:,:], int32[:], float64[:,:], float64[:,:], int32, float64[:,:], float64[:,:], float64[:,:], float64[:], float64[:,:], int32[:])
(int32[:,:], float64[:,:], float64[:,:])
(int32[:,:], int32[:,:], int32[:,:], int32[:], int32[:], int32[:], int32, int32, int32, int32)
(float64[:,:,:], int32[:,:])
(float64[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], float64[:,:], float64[:,:], float64[:,:], float64[:,:], float64[:], float64[:], float64[:], float64[:], float64[:], float64[:], int32[:], float64[:,:])
(int32[:,:], int32[

In [38]:
  d_cells = domain._cells._nodeid
  d_points = domain._nodes._vertex
  d_center = domain._cells._center
  d_volume = domain._cells._volume

In [23]:
d_cells

array([[  6,  62, 106,  61,   4],
       [ 62, 106,  61, 520,   4],
       [ 61, 520, 106, 431,   4],
       ...,
       [358, 521,  89,  43,   4],
       [521,   8,  89,  43,   4],
       [ 89,   8,   0,  43,   4]], dtype=int32)

In [13]:
cube = np.array([
  [0, 0, 0],
  [1, 1, 1],
  [2, 2, 2],
  [3, 3, 3],
  [4, 4, 4],
  [5, 5, 5],
  [6, 6, 6],
  [7, 7, 7],
], dtype=np.float64)
points = np.array([
  [cube[6], cube[7], cube[2], cube[5]],
  [cube[7], cube[2], cube[5], cube[4]],
  [cube[5], cube[4], cube[2], cube[1]],
  [cube[7], cube[2], cube[4], cube[3]],
  [cube[2], cube[1], cube[4], cube[3]],
  [cube[4], cube[1], cube[0], cube[3]],
])

points


array([[[6., 6., 6.],
        [7., 7., 7.],
        [2., 2., 2.],
        [5., 5., 5.]],

       [[7., 7., 7.],
        [2., 2., 2.],
        [5., 5., 5.],
        [4., 4., 4.]],

       [[5., 5., 5.],
        [4., 4., 4.],
        [2., 2., 2.],
        [1., 1., 1.]],

       [[7., 7., 7.],
        [2., 2., 2.],
        [4., 4., 4.],
        [3., 3., 3.]],

       [[2., 2., 2.],
        [1., 1., 1.],
        [4., 4., 4.],
        [3., 3., 3.]],

       [[4., 4., 4.],
        [1., 1., 1.],
        [0., 0., 0.],
        [3., 3., 3.]]])

In [65]:
def main_test(domain, decimal_precision):
  """
  :param domain: manapy domain
  :param decimal_precision: decimal precision for comparison

  :details Create a cuboid mesh constructed from cuboid cells, retrieve the cell center and area, and compare them with the corresponding domain values.
  """
  With = 10.0
  Height = 5.0
  Depth = 15.0
  StepX = With / 10.0
  StepY = Height / 10.0
  StepZ = Depth / 10.0

  d_cells = domain._cells._nodeid
  d_points = domain._nodes._vertex[:, 0:3]
  d_center = domain._cells._center[:, 0:3]
  d_volume = domain._cells._volume

  cmp = 0
  for x in np.arange(With, 0.0, -StepX):
    for z in np.arange(Depth, 0.0, -StepZ):
      for y in np.arange(Height, 0.0, -StepY):
        cuboid_points = np.array([
          [x, y, z],
          [x - StepX, y, z],
          [x - StepX, y, z - StepZ],
          [x, y, z - StepZ],

          [x, y - StepY, z],
          [x - StepX, y - StepY, z],
          [x - StepX, y - StepY, z - StepZ],
          [x, y - StepY, z - StepZ],
        ], dtype=d_points.dtype)

        tetrahedrons = np.array([
          [cuboid_points[0], cuboid_points[1], cuboid_points[3], cuboid_points[4]],
          [cuboid_points[1], cuboid_points[3], cuboid_points[4], cuboid_points[5]],
          [cuboid_points[4], cuboid_points[5], cuboid_points[3], cuboid_points[7]],
          [cuboid_points[1], cuboid_points[3], cuboid_points[5], cuboid_points[2]],
          [cuboid_points[3], cuboid_points[7], cuboid_points[5], cuboid_points[2]],
          [cuboid_points[5], cuboid_points[7], cuboid_points[6], cuboid_points[2]],
        ])
        if cmp == 1:
          print(tetrahedrons)
        cmp += 1

main_test(domain, 4)

[[[10.   4.5 15. ]
  [ 9.   4.5 15. ]
  [10.   4.5 13.5]
  [10.   4.  15. ]]

 [[ 9.   4.5 15. ]
  [10.   4.5 13.5]
  [10.   4.  15. ]
  [ 9.   4.  15. ]]

 [[10.   4.  15. ]
  [ 9.   4.  15. ]
  [10.   4.5 13.5]
  [10.   4.  13.5]]

 [[ 9.   4.5 15. ]
  [10.   4.5 13.5]
  [ 9.   4.  15. ]
  [ 9.   4.5 13.5]]

 [[10.   4.5 13.5]
  [10.   4.  13.5]
  [ 9.   4.  15. ]
  [ 9.   4.5 13.5]]

 [[ 9.   4.  15. ]
  [10.   4.  13.5]
  [ 9.   4.  13.5]
  [ 9.   4.5 13.5]]]


In [39]:
m_mesh.cells['tetra'][0:6]

array([[  6,  62, 106,  61],
       [ 62, 106,  61, 520],
       [ 61, 520, 106, 431],
       [ 62, 106, 520, 197],
       [106, 431, 520, 197],
       [520, 431, 602, 197]])

In [36]:
m_mesh.cells['hexahedron'][0]

array([  6,  62, 197, 106,  61, 520, 602, 431])

In [59]:
ids = [6,  62, 197, 106,  61, 520, 602, 431]
id_to_idx = {item: idx for idx, item in enumerate(ids)}
id_to_idx

{6: 0, 62: 1, 197: 2, 106: 3, 61: 4, 520: 5, 602: 6, 431: 7}

In [60]:
np.vectorize(lambda x : id_to_idx.get(x))(m_mesh.cells['tetra'][0:6])

array([[0, 1, 3, 4],
       [1, 3, 4, 5],
       [4, 5, 3, 7],
       [1, 3, 5, 2],
       [3, 7, 5, 2],
       [5, 7, 6, 2]])

In [68]:
m_mesh.points[m_mesh.cells['tetra'][11]]

array([[ 9. ,  4. , 15. ],
       [10. ,  4. , 13.5],
       [ 9. ,  4. , 13.5],
       [ 9. ,  4.5, 13.5]])

In [42]:
d_cells[0][:-1]

array([  6,  62, 197, 106,  61, 520, 602, 431], dtype=int32)

In [47]:
c_points = d_points[d_cells[0][:-1]][:, 0:3]
print(c_points)

[[10.   5.  15. ]
 [ 9.   5.  15. ]
 [ 9.   5.  13.5]
 [10.   5.  13.5]
 [10.   4.5 15. ]
 [ 9.   4.5 15. ]
 [ 9.   4.5 13.5]
 [10.   4.5 13.5]]


In [32]:
def b():
  With = 10.0
  Height = 5.0
  Depth = 15.0
  StepX = With / 10.0
  StepY = Height / 10.0
  StepZ = Depth / 10.0

  cmp = 0
  for x in np.arange(With, 0.0, -StepX):
    for z in np.arange(Depth, 0.0, -StepZ):
      for y in np.arange(Height, 0.0, -StepY):
        points = np.array([
          [x, y, z],
          [x - StepX, y, z],
          [x - StepX, y, z - StepZ],
          [x, y, z - StepZ],

          [x, y - StepY, z],
          [x - StepX, y - StepY, z],
          [x - StepX, y - StepY, z - StepZ],
          [x, y - StepY, z - StepZ],
        ], dtype=d_points.dtype)
        m_points= m_mesh.points[m_mesh.cells['hexahedron'][cmp]]
        np.testing.assert_almost_equal(points, m_points)
        cmp = cmp + 1

b()

In [57]:
d_cells[[0, 1]][:, :-1]

array([[  6,  62, 197, 106,  61, 520, 602, 431],
       [ 61, 520, 602, 431,  60, 519, 603, 422]], dtype=int32)

In [29]:
m_mesh.points[m_mesh.cells['hexahedron'][[998, 999]]]

array([[[1. , 1. , 1.5],
        [0. , 1. , 1.5],
        [0. , 1. , 0. ],
        [1. , 1. , 0. ],
        [1. , 0.5, 1.5],
        [0. , 0.5, 1.5],
        [0. , 0.5, 0. ],
        [1. , 0.5, 0. ]],

       [[1. , 0.5, 1.5],
        [0. , 0.5, 1.5],
        [0. , 0.5, 0. ],
        [1. , 0.5, 0. ],
        [1. , 0. , 1.5],
        [0. , 0. , 1.5],
        [0. , 0. , 0. ],
        [1. , 0. , 0. ]]])

In [69]:
d_cells = domain._cells._nodeid
d_points = domain._nodes._vertex[:, 0:3]
d_center = domain._cells._center[:, 0:3]
d_volume = domain._cells._volume

In [72]:
d_volume

array([0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125])